In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

In [3]:
path = './data.csv'
cancerData = pd.read_csv(path)
cancerData = cancerData.drop(1733)

## Recurrence Rate

In [4]:
temp = cancerData['是否复发（是1 否0）'].fillna(0)
rate = sum(temp)/(len(temp))
rate

0.5626081938834391

In [5]:
# early recurrence
temp = cancerData['是否为早期复发'].fillna(0)
rate = sum(temp)/len(temp)
rate

0.44143104443162146

# Grouping Non-Numeric Data Value

In [6]:
textIdx = cancerData['严重并发症']!='0'
cancerData['严重并发症'][textIdx] = 1
cancerData.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,性别（1男0女）,年龄,肿瘤大小a（cm）,肿瘤大小b（cm）,肿瘤大小c（cm）,肿瘤数量,肝硬化类型（0无乙肝1丙肝2酒精3其他4）,肝外转移（无0有1）,Child分级,...,术前低密度脂蛋白,消融时间（s）,功率（W）,酒精消融（有1无0）,并发症（有1无0）,严重并发症,完全消融（是1否0）,是否复发（是1 否0）,随访时间(单位：月),是否为早期复发
0,1.0,1,39,2.5,2.2,NaN,3.0,1.0,0.0,A,...,NaN,1020.0,50.0,0.0,0.0,0,1.0,1.0,33.80,0.0
1,2.0,1,58,4.0,2.3,NaN,2.0,1.0,0.0,A,...,1.84,840.0,50.0,1.0,1.0,0,1.0,1.0,20.00,1.0
2,3.0,1,47,2.5,2.1,NaN,4.0,0.0,1.0,A,...,NaN,480.0,50.0,0.0,0.0,0,1.0,1.0,2.57,1.0
3,4.0,1,53,3.4,3.2,NaN,28.0,0.0,0.0,A,...,NaN,660.0,50.0,0.0,1.0,0,1.0,1.0,2.33,1.0
4,5.0,0,46,3.6,2.9,NaN,1.0,1.0,0.0,B,...,NaN,780.0,50.0,1.0,1.0,0,1.0,0.0,100.97,0.0
5,6.0,1,53,2.4,2.1,NaN,1.0,1.0,0.0,B,...,NaN,510.0,50.0,0.0,0.0,0,1.0,1.0,61.90,0.0
6,7.0,1,65,6.2,3.9,NaN,1.0,1.0,0.0,B,...,NaN,2250.0,60.0,1.0,1.0,1,1.0,1.0,5.37,1.0
7,8.0,1,58,3.5,3.0,NaN,1.0,1.0,0.0,A,...,NaN,870.0,50.0,0.0,0.0,0,1.0,1.0,0.87,1.0
8,9.0,1,39,1.9,1.8,NaN,3.0,0.0,0.0,A,...,NaN,690.0,40.0,1.0,0.0,0,1.0,1.0,10.43,1.0
9,10.0,1,39,1.9,1.9,NaN,2.0,1.0,0.0,A,...,NaN,420.0,0.0,1.0,0.0,0,1.0,1.0,65.93,0.0


In [7]:
cancerData['Child分级'][cancerData['Child分级']=='A'] = 0
cancerData['Child分级'][cancerData['Child分级']=='B'] = 1
cancerData['Child分级'][cancerData['Child分级']=='C'] = 2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we c

In [8]:
cancerData = cancerData.apply(pd.to_numeric,errors = 'coerce')
cancerData.dtypes

ID                       float64
性别（1男0女）                   int64
年龄                       float64
肿瘤大小a（cm）                float64
肿瘤大小b（cm）                float64
肿瘤大小c（cm）                float64
肿瘤数量                     float64
肝硬化类型（0无乙肝1丙肝2酒精3其他4）    float64
肝外转移（无0有1）               float64
Child分级                  float64
病理（高分化0中1低2无3）           float64
腹水（无0有1）                 float64
血管癌栓（无0有1）               float64
术前ALT                    float64
术前AST                    float64
术前GT                     float64
术前AFP                    float64
术前碱性磷酸酶                  float64
术前总蛋白                    float64
术前白蛋白                    float64
术前总胆红素                   float64
术前直胆红素                   float64
术前肌酐                     float64
术前血糖                     float64
术前胆碱酯酶                   float64
术前血红蛋白                   float64
术前红细胞                    float64
术前血小板                    float64
术前淋巴细胞                   float64
术前白细胞                    float64
术前中性粒细胞   

# Fill in Missing Value with Mean

## Number of Missing Values

In [9]:
cancerData.isnull().sum()/len(cancerData)

ID                       0.000000
性别（1男0女）                 0.000000
年龄                       0.001731
肿瘤大小a（cm）                0.013272
肿瘤大小b（cm）                0.024235
肿瘤大小c（cm）                0.702827
肿瘤数量                     0.001154
肝硬化类型（0无乙肝1丙肝2酒精3其他4）    0.007501
肝外转移（无0有1）               0.000000
Child分级                  0.001154
病理（高分化0中1低2无3）           0.000000
腹水（无0有1）                 0.000000
血管癌栓（无0有1）               0.000000
术前ALT                    0.203116
术前AST                    0.203116
术前GT                     0.287940
术前AFP                    0.289671
术前碱性磷酸酶                  0.306982
术前总蛋白                    0.393537
术前白蛋白                    0.234853
术前总胆红素                   0.221004
术前直胆红素                   0.226197
术前肌酐                     0.273514
术前血糖                     0.267167
术前胆碱酯酶                   0.333526
术前血红蛋白                   0.232545
术前红细胞                    0.244662
术前血小板                    0.215234
术前淋巴细胞                   0.241777
术前白细胞         

In [10]:
# delete columns with high missing rate
cancerData.drop(columns = ['术前低密度脂蛋白','术前高密度脂蛋白','术前胆固醇','术前甘油三酯','肿瘤大小c（cm）'],inplace = True)

## Find Columns with Missing Value

In [11]:
nullIdx = cancerData.isnull().any(axis = 1)
temp = cancerData[nullIdx==False]
cancerData = temp

In [12]:
cancerData.index = range(len(temp))
cancerData

,ID,性别（1男0女）,年龄,肿瘤大小a（cm）,肿瘤大小b（cm）,肿瘤数量,肝硬化类型（0无乙肝1丙肝2酒精3其他4）,肝外转移（无0有1）,Child分级,病理（高分化0中1低2无3）,...,术前INR,消融时间（s）,功率（W）,酒精消融（有1无0）,并发症（有1无0）,严重并发症,完全消融（是1否0）,是否复发（是1 否0）,随访时间(单位：月),是否为早期复发
0,1.0,1,39.0,2.5,2.2,3.0,1.0,0.0,0.0,3.0,...,1.03,1020.0,50.0,0.0,0.0,0,1.0,1.0,33.80,0.0
1,2.0,1,58.0,4.0,2.3,2.0,1.0,0.0,0.0,3.0,...,0.92,840.0,50.0,1.0,1.0,0,1.0,1.0,20.00,1.0
2,3.0,1,47.0,2.5,2.1,4.0,0.0,1.0,0.0,3.0,...,0.82,480.0,50.0,0.0,0.0,0,1.0,1.0,2.57,1.0
3,16.0,1,66.0,1.9,1.5,26.0,1.0,0.0,0.0,3.0,...,0.97,240.0,50.0,0.0,1.0,0,1.0,1.0,9.93,1.0
4,17.0,1,40.0,2.0,1.6,1.0,1.0,0.0,0.0,3.0,...,1.02,600.0,50.0,0.0,1.0,0,1.0,1.0,19.50,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,1726.0,1,71.0,2.1,1.7,2.0,2.0,0.0,0.0,1.0,...,1.07,360.0,50.0,0.0,0.0,0,1.0,1.0,15.43,1.0
641,1727.0,1,77.0,4.2,3.9,3.0,1.0,0.0,1.0,3.0,...,1.06,1740.0,60.0,0.0,1.0,0,1.0,1.0,3.17,1.0
642,1728.0,1,64.0,3.8,2.9,2.0,1.0,0.0,0.0,3.0,...,1.26,480.0,60.0,0.0,1.0,0,1.0,0.0,29.97,0.0
643,1731.0,1,50.0,4.9,4.5,1.0,1.0,0.0,0.0,3.0,...,1.11,1080.0,50.0,0.0,1.0,0,1.0,0.0,30.23,0.0


In [13]:
temp = pd.concat([cancerData,pd.get_dummies(cancerData['肝硬化类型（0无乙肝1丙肝2酒精3其他4）'])],axis = 1)
cancerData = temp

In [14]:
cancerData.rename(columns = {0.0:'无肝硬化',1.0:'乙肝',2.0:'丙肝',3.0:'酒精肝',4.0:'其他'},inplace = True)
cancerData.drop(columns = '其他',inplace = True)

In [15]:
sum(cancerData.isnull().any()) == 0

True

In [16]:
plt.rcParams['font.sans-serif'] = ['SimHei'] # 步骤一（替换sans-serif字体）
plt.rcParams['axes.unicode_minus'] = False

In [17]:
# dataShape = cancerData.shape
# for i in range(dataShape[1]):
#     cancerData.iloc[:,i].plot.box()
#     plt.savefig(str(i)+'.pdf')
#     plt.show()

## Checking Existence of Inconsistency

In [16]:
cancerData.head(10)

,ID,性别（1男0女）,年龄,肿瘤大小a（cm）,肿瘤大小b（cm）,肿瘤数量,肝硬化类型（0无乙肝1丙肝2酒精3其他4）,肝外转移（无0有1）,Child分级,病理（高分化0中1低2无3）,...,并发症（有1无0）,严重并发症,完全消融（是1否0）,是否复发（是1 否0）,随访时间(单位：月),是否为早期复发,无肝硬化,乙肝,丙肝,酒精肝
0,1.0,1,39.0,2.5,2.2,3.0,1.0,0.0,0.0,3.0,...,0.0,0,1.0,1.0,33.80,0.0,0,1,0,0
1,2.0,1,58.0,4.0,2.3,2.0,1.0,0.0,0.0,3.0,...,1.0,0,1.0,1.0,20.00,1.0,0,1,0,0
2,3.0,1,47.0,2.5,2.1,4.0,0.0,1.0,0.0,3.0,...,0.0,0,1.0,1.0,2.57,1.0,1,0,0,0
3,16.0,1,66.0,1.9,1.5,26.0,1.0,0.0,0.0,3.0,...,1.0,0,1.0,1.0,9.93,1.0,0,1,0,0
4,17.0,1,40.0,2.0,1.6,1.0,1.0,0.0,0.0,3.0,...,1.0,0,1.0,1.0,19.50,1.0,0,1,0,0
5,18.0,1,55.0,2.6,1.9,1.0,1.0,0.0,0.0,1.0,...,0.0,0,1.0,1.0,4.80,1.0,0,1,0,0
6,19.0,1,51.0,3.5,2.8,2.0,1.0,0.0,0.0,1.0,...,1.0,0,1.0,0.0,30.90,0.0,0,1,0,0
7,22.0,1,66.0,2.5,2.1,2.0,1.0,0.0,0.0,3.0,...,0.0,0,1.0,1.0,20.37,1.0,0,1,0,0
8,27.0,1,58.0,6.4,4.5,3.0,1.0,1.0,0.0,3.0,...,1.0,0,1.0,0.0,4.23,0.0,0,1,0,0
9,28.0,1,61.0,2.3,1.9,1.0,1.0,0.0,0.0,3.0,...,0.0,0,1.0,0.0,42.17,0.0,0,1,0,0


In [17]:
temp = cancerData['是否复发（是1 否0）'] - cancerData['是否为早期复发']
idx = temp>=0
sum(idx) == len(temp)

True

## Data Normalization

In [18]:
visitTime = cancerData['随访时间(单位：月)']
earlyRecur = cancerData['是否为早期复发']

In [19]:
cancerData.drop(columns = ['ID','是否复发（是1 否0）','随访时间(单位：月)','是否为早期复发','肝硬化类型（0无乙肝1丙肝2酒精3其他4）'],inplace = True)
cancerData

,性别（1男0女）,年龄,肿瘤大小a（cm）,肿瘤大小b（cm）,肿瘤数量,肝外转移（无0有1）,Child分级,病理（高分化0中1低2无3）,腹水（无0有1）,血管癌栓（无0有1）,...,消融时间（s）,功率（W）,酒精消融（有1无0）,并发症（有1无0）,严重并发症,完全消融（是1否0）,无肝硬化,乙肝,丙肝,酒精肝
0,1,39.0,2.5,2.2,3.0,0.0,0.0,3.0,0.0,0.0,...,1020.0,50.0,0.0,0.0,0,1.0,0,1,0,0
1,1,58.0,4.0,2.3,2.0,0.0,0.0,3.0,0.0,0.0,...,840.0,50.0,1.0,1.0,0,1.0,0,1,0,0
2,1,47.0,2.5,2.1,4.0,1.0,0.0,3.0,0.0,0.0,...,480.0,50.0,0.0,0.0,0,1.0,1,0,0,0
3,1,66.0,1.9,1.5,26.0,0.0,0.0,3.0,0.0,0.0,...,240.0,50.0,0.0,1.0,0,1.0,0,1,0,0
4,1,40.0,2.0,1.6,1.0,0.0,0.0,3.0,0.0,0.0,...,600.0,50.0,0.0,1.0,0,1.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,1,71.0,2.1,1.7,2.0,0.0,0.0,1.0,0.0,0.0,...,360.0,50.0,0.0,0.0,0,1.0,0,0,1,0
641,1,77.0,4.2,3.9,3.0,0.0,1.0,3.0,0.0,0.0,...,1740.0,60.0,0.0,1.0,0,1.0,0,1,0,0
642,1,64.0,3.8,2.9,2.0,0.0,0.0,3.0,0.0,0.0,...,480.0,60.0,0.0,1.0,0,1.0,0,1,0,0
643,1,50.0,4.9,4.5,1.0,0.0,0.0,3.0,0.0,0.0,...,1080.0,50.0,0.0,1.0,0,1.0,0,1,0,0


In [20]:
cancerDataNorm = (cancerData-cancerData.mean())/cancerData.std()
cancerDataNorm['无肝硬化'] = cancerData['无肝硬化'].astype(np.int8)
cancerDataNorm['乙肝'] = cancerData['乙肝'].astype(np.int8)
cancerDataNorm['丙肝'] = cancerData['丙肝'].astype(np.int8)
cancerDataNorm['酒精肝'] = cancerData['酒精肝'].astype(np.int8)
cancerDataNorm['Child分级'] = cancerData['Child分级'].astype(np.int8)
cancerDataNorm['肝外转移（无0有1）'] = cancerData['肝外转移（无0有1）'].astype(np.int8)
cancerDataNorm['病理（高分化0中1低2无3）'] = cancerData['病理（高分化0中1低2无3）'].astype(np.int8)
cancerDataNorm['腹水（无0有1）'] = cancerData['腹水（无0有1）'].astype(np.int8)
cancerDataNorm['血管癌栓（无0有1）'] = cancerData['血管癌栓（无0有1）'].astype(np.int8)
cancerDataNorm['酒精消融（有1无0）'] = cancerData['酒精消融（有1无0）'].astype(np.int8)
cancerDataNorm['并发症（有1无0）'] = cancerDataNorm['并发症（有1无0）'].astype(np.int8)
cancerDataNorm['严重并发症'] = cancerData['严重并发症'].astype(np.int8)
cancerDataNorm['完全消融（是1否0）'] = cancerData['完全消融（是1否0）'].astype(np.int8)
cancerDataNorm['性别（1男0女）'] = cancerData['性别（1男0女）'].astype(np.int8)

In [21]:
cancerDataNorm

,性别（1男0女）,年龄,肿瘤大小a（cm）,肿瘤大小b（cm）,肿瘤数量,肝外转移（无0有1）,Child分级,病理（高分化0中1低2无3）,腹水（无0有1）,血管癌栓（无0有1）,...,消融时间（s）,功率（W）,酒精消融（有1无0）,并发症（有1无0）,严重并发症,完全消融（是1否0）,无肝硬化,乙肝,丙肝,酒精肝
0,1,-1.804675,-0.281077,-0.182246,0.121174,0,0,3,0,0,...,1.029620,-0.444295,0,-1,0,1,0,1,0,0
1,1,-0.075083,0.738579,-0.098343,-0.242348,0,0,3,0,0,...,0.594632,-0.444295,1,0,0,1,0,1,0,0
2,1,-1.076425,-0.281077,-0.266150,0.484695,1,0,3,0,0,...,-0.275342,-0.444295,0,-1,0,1,1,0,0,0
3,1,0.653166,-0.688940,-0.769571,8.482164,0,0,3,0,0,...,-0.855325,-0.444295,0,0,0,1,0,1,0,0
4,1,-1.713643,-0.620963,-0.685667,-0.605869,0,0,3,0,0,...,0.014649,-0.444295,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,1,1.108322,-0.552986,-0.601764,-0.242348,0,0,1,0,0,...,-0.565334,-0.444295,0,-1,0,1,0,0,1,0
641,1,1.654508,0.874533,1.244113,0.121174,0,1,3,0,0,...,2.769569,1.226667,0,0,0,1,0,1,0,0
642,1,0.471104,0.602625,0.405078,-0.242348,0,0,3,0,0,...,-0.275342,1.226667,0,0,0,1,0,1,0,0
643,1,-0.803332,1.350373,1.747534,-0.605869,0,0,3,0,0,...,1.174615,-0.444295,0,0,0,1,0,1,0,0


## Data Reduction

## PCA

In [22]:
pca = PCA(n_components=0.95)
cancerDataNormPCA = pca.fit_transform(cancerDataNorm)

In [23]:
cancerDataNormPCA[1]

array([-2.11247914,  0.85994024,  0.31404753,  0.17349855,  1.8325834 ,
        0.48127236,  0.64009395, -0.47515978, -0.19998834,  0.36491567,
        0.81564534,  0.52745871, -0.58546102,  0.44521417, -0.71358679,
        0.28073465,  0.25552503, -0.45303041, -0.13913761, -0.78260975,
        1.20600946,  0.19499538, -0.43251884, -0.1961547 ,  0.338279  ])

# Model Training and Prediction

## Logistic Regression

In [30]:
X = cancerDataNormPCA
y = earlyRecur.astype(np.int8)

In [31]:
lrClassifier = LogisticRegression(penalty = 'l2')
kf = KFold(n_splits=5)
acc = list()
auc = list()
for train_index,test_index in kf.split(X):
    
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    train_length = len(y_train)
    if train_length >= 2*sum(y_train):
        smo = SMOTE(sampling_strategy = {1:train_length-sum(y_train)},random_state = 100)
    else:
        smo = SMOTE(sampling_strategy = {0:sum(y_train)},random_state = 100)
    X_smo,y_smo = smo.fit_sample(X_train,y_train)
    lrClassifier.fit(X_smo,y_smo)
    y_pred = lrClassifier.predict(X_test)
    acc.append(accuracy_score(y_test,y_pred))
    auc.append(roc_auc_score(y_test,y_pred))
np.mean(acc),np.mean(auc)

(0.7085271317829458, 0.6937369932534263)

## SVM

In [32]:
from sklearn import svm

In [33]:
SVMClassifier=svm.SVC(kernel='rbf',gamma=0.02,decision_function_shape='ovo',C=0.6)
kf = KFold(n_splits=5)
acc = list()
auc = list()
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    train_length = len(y_train)
    if train_length >= 2*sum(y_train):
        smo = SMOTE(sampling_strategy = {1:train_length-sum(y_train)},random_state = 100)
    else:
        smo = SMOTE(sampling_strategy = {0:sum(y_train)},random_state = 100)
    X_smo,y_smo = smo.fit_sample(X_train,y_train)
    SVMClassifier.fit(X_smo,y_smo)
    y_pred = SVMClassifier.predict(X_test)
    acc.append(accuracy_score(y_test,y_pred))
    auc.append(roc_auc_score(y_test,y_pred))
np.mean(acc),np.mean(auc)

(0.6899224806201552, 0.6549200857452053)

# Decision Tree

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
DClassifier=DecisionTreeClassifier()
kf = KFold(n_splits=5)
acc = list()
auc = list()
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    train_length = len(y_train)
    if train_length >= 2*sum(y_train):
        smo = SMOTE(sampling_strategy = {1:train_length-sum(y_train)},random_state = 100)
    else:
        smo = SMOTE(sampling_strategy = {0:sum(y_train)},random_state = 100)
    X_smo,y_smo = smo.fit_sample(X_train,y_train)
    DClassifier.fit(X_smo,y_smo)
    y_pred = DClassifier.predict(X_test)
    acc.append(accuracy_score(y_test,y_pred))
    auc.append(roc_auc_score(y_test,y_pred))
np.mean(acc),np.mean(auc)

(0.5922480620155038, 0.5771392142559549)

# Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
RFClassifier=RandomForestClassifier()
kf = KFold(n_splits=5)
acc = list()
auc = list()
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    train_length = len(y_train)
    if train_length >= 2*sum(y_train):
        smo = SMOTE(sampling_strategy = {1:train_length-sum(y_train)},random_state = 100)
    else:
        smo = SMOTE(sampling_strategy = {0:sum(y_train)},random_state = 100)
    X_smo,y_smo = smo.fit_sample(X_train,y_train)
    RFClassifier.fit(X_smo,y_smo)
    y_pred = RFClassifier.predict(X_test)
    acc.append(accuracy_score(y_test,y_pred))
    auc.append(roc_auc_score(y_test,y_pred))
np.mean(acc),np.mean(auc)

(0.710077519379845, 0.6802395996850122)

# Perceptron

In [38]:
from sklearn.linear_model import Perceptron

In [39]:
PClassifier=Perceptron(max_iter = 1000, tol = None)
kf = KFold(n_splits=5)
acc = list()
auc = list()
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    train_length = len(y_train)
    if train_length >= 2*sum(y_train):
        smo = SMOTE(sampling_strategy = {1:train_length-sum(y_train)},random_state = 100)
    else:
        smo = SMOTE(sampling_strategy = {0:sum(y_train)},random_state = 100)
    X_smo,y_smo = smo.fit_sample(X_train,y_train)
    PClassifier.fit(X_smo,y_smo)
    y_pred = PClassifier.predict(X_test)
    acc.append(accuracy_score(y_test,y_pred))
    auc.append(roc_auc_score(y_test,y_pred))
np.mean(acc),np.mean(auc)

(0.6093023255813954, 0.5878794673134824)